# Tracing pluripotency of human early embryos and embryonic stem cells by single cell RNA-seq

*Public on Aug 10, 2013*

**Homo sapiens**: Expression profiling by high throughput sequencing

*Find the casual relationship between gene expression network and cellular phenotype at single cell resolution. We collected donated human pre-implatation embryos, and the embryonic stem cells derived from them, isolate individual cells, prepared single cell cDNAs, and sequenced them by HiSeq2000. Then we analyzed the expression of known RefSeq genes.*
 	
Overall design	We get transcriptome of 124 individual cells from human pre-implantation embryos and human embryonic stem cells by applying single cell RNA-seq technique we recently developed[1][2][3][4]. We did in-depth bioinformatic analysis to these data and found very dynamic expression of protein-coding genes.
[1] Tang, F. et al. (2010a) Tracing the Derivation of Embryonic Stem Cells from the Inner Cell Mass by Single-Cell RNA-Seq Analysis. Cell Stem Cell 6, 468-478.
[2] Tang, F. et al. (2010b) RNA-Seq analysis to capture the transcriptome landscape of a single cell. Nat Protocols 5, 516-535.
[3] Tang, F. et al. (2009) mRNA-Seq whole-transcriptome analysis of a single cell. Nat Meth 6, 377-382.
[4] Tang, F. et al. (2011) Development and applications of single-cell transcriptome analysis. Nat Meth 8, S6-S11.
 	
Contributor(s)	*Tang F, Qiao J, Li R*
Citation(s)	

Yan L, Yang M, Guo H, Yang L et al. Single-cell RNA-Seq profiling of human preimplantation embryos and embryonic stem cells. Nat Struct Mol Biol 2013 Sep;20(9):1131-9. PMID: 23934149

In [1]:
try:
    from urllib2 import urlopen
    from urllib2 import URLError
except ImportError:
    from urllib.request import urlopen
    from urllib.error import URLError

In [2]:
import os
from contextlib import closing

In [3]:
from GPy.util.datasets import download_url
from GPy.util import datasets
import pandas as pd, numpy as np
from pandas import read_csv

## Download the data
The info is in the Series GSE36552, thus we will download that series first:

In [4]:
url = 'ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE36nnn/GSE36552/matrix/GSE36552_series_matrix.txt.gz'
_saved = datasets.data_path
dirname = 'GSE36552'

try:
    datasets.data_path = './'
    if True:#not os.path.exists(filename):
        download_url(url, dirname)                   
finally:
    datasets.data_path = _saved

GSE36552_series_matrix.txt.gz
[==============================]   0.009/0.009MB


Now we have the supplementary links in the info row *!Sample_supplementary_file_3*. This allows us to download all the gene expression (RPKM) values for all samples:

In [5]:
try:
    datasets.data_path = './'
    info = read_csv(os.path.join(dirname, 'GSE36552_series_matrix.txt.gz'), sep='\t', skiprows=52, compression='gzip', header=0, index_col=0)
    for suburl in info.loc['!Sample_supplementary_file_3']:
        download_url(suburl, dirname)
finally:
    datasets.data_path = _saved

GSM896803_Sample_M21.expression.txt.gz
[==============================]   0.248/0.248MB
GSM896804_Sample_M22.expression.txt.gz
[==============================]   0.249/0.249MB
GSM896805_Sample_M23.expression.txt.gz
[==============================]   0.252/0.252MB
GSM896806_Sample_C1.expression.txt.gz
[==============================]   0.256/0.256MB
GSM896807_Sample_C2.expression.txt.gz
[==============================]   0.249/0.249MB
GSM896808_Sample_C3.expression.txt.gz
[==============================]   0.251/0.251MB
GSM896809_Sample_B1.expression.txt.gz
[==============================]   0.258/0.258MB
GSM896810_Sample_B2.expression.txt.gz
[==============================]   0.257/0.257MB
GSM896811_Sample_B3.expression.txt.gz
[==============================]   0.254/0.254MB
GSM896812_Sample_B4.expression.txt.gz
[==============================]   0.253/0.253MB
GSM896813_Sample_B5.expression.txt.gz
[==============================]   0.252/0.252MB
GSM896814_Sample_B6.expression.txt.gz
[=

### Collect the data together into one DataFrame:

In [6]:
data = None
for name, col in info.iterkv():
    filename = os.path.join(dirname, os.path.split(col['!Sample_supplementary_file_3'])[1])
    #header = read_csv(filename, compression='gzip', index_col=0, header=None, nrows=1, sep=' ', skipinitialspace=True)
    _tmp = read_csv(filename, skiprows=1, compression='gzip', sep='\t', header=None, index_col=0, usecols=[0,4], iterator=False)
    if data is None:
        data = _tmp
        data.columns = [name]
        data.index.name = None
    else:
        data[name] = _tmp
data = data.T

### Make the index hierarchical

In [7]:
def process_index(k):
    splits = k.rsplit(' ', 2)
    return splits

In [8]:
data.index = pd.MultiIndex.from_tuples([process_index(k) for k,v in data.iterrows()])

### Filter the data for coverage and detection

In [120]:
Y = data.copy()
print "before filtering: {}".format(Y.shape)
# omit cells without detection
fil = (Y.sum(1)>0).values
Y = Y.loc[fil, :]
# omit genes with too low coverage
fil = (Y.sum(0)>1e3).values
Y  = Y.loc[:, fil] 
# omit genes with too high coverage
fil = (np.log1p(Y)>4.5).values
Y  = Y.loc[:, (~fil.any(0))] 
print "after filtering: {}".format(Y.shape)
#Y = Y.reset_index().set_index('index', append=True)
#Y['labels'] = labels.values
#Y = Y.set_index('labels', append=True)
#Y = Y.reorder_levels([0,2,1])
#Y = Y.reset_index([0,2]).loc[ulabels].set_index(['level_0', 'index'], append=True)

before filtering: (124, 14514)
after filtering: (124, 1955)


In [9]:
data.to_csv('singlecellTang.csv')